## Loads the ROMS data and converts from sigma coordinate to meters.  Saves as NetCDF for input into vertical climate velocity R Script

In [20]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [4]:
roms = xr.open_dataset('/Users/nyelab/Downloads/DopAnV2R3-ini2007_da_mon_avg.nc')

In [5]:
roms

<xarray.Dataset>
Dimensions:     (eta_rho: 63, eta_u: 63, eta_v: 62, ocean_time: 168, s_rho: 40, xi_rho: 76, xi_u: 76, xi_v: 77)
Coordinates:
  * ocean_time  (ocean_time) datetime64[ns] 2007-01-17 2007-02-15 ... 2020-12-16
  * s_rho       (s_rho) float64 -0.9875 -0.9625 -0.9375 ... -0.0375 -0.0125
    lat_rho     (eta_rho, xi_rho) float64 ...
    lon_rho     (eta_rho, xi_rho) float64 ...
    lat_u       (eta_u, xi_u) float64 ...
    lon_u       (eta_u, xi_u) float64 ...
    lat_v       (eta_v, xi_v) float64 ...
    lon_v       (eta_v, xi_v) float64 ...
Dimensions without coordinates: eta_rho, eta_u, eta_v, xi_rho, xi_u, xi_v
Data variables:
    salt        (ocean_time, s_rho, eta_rho, xi_rho) float64 ...
    Cs_r        (s_rho) float64 ...
    zeta        (ocean_time, eta_rho, xi_rho) float64 ...
    h           (eta_rho, xi_rho) float64 ...
    hc          float64 ...
    temp        (ocean_time, s_rho, eta_rho, xi_rho) float64 ...
    u           (ocean_time, s_rho, eta_u, xi_u) float64 ...
    v           (ocean_time, s_rho, eta_v, xi_v) float64 ...
Attributes: (12/48)
    file:                       doppio_avg_0366.nc
    format:                     netCDF-4/HDF5 file
    Conventions:                CF-1.4, SGRID-0.3
    type:                       ROMS/TOMS nonlinear model averages file
    title:                      DopAnV2R3-ini2007 - ROMS DOPPIO 7km reanalysi...
    var_info:                   ../Data/varinfo809t_daily.dat
    ...                         ...
    summary:                    doppio
    History:                    Translated to CF-1.0 Conventions by Netcdf-Ja...
    geospatial_lat_min:         37.178910308832386
    geospatial_lat_max:         43.17683923036529
    geospatial_lon_min:         -76.23060799706121
    geospatial_lon_max:         -68.17979698921016

In [7]:
# the following transformation is from: http://xarray.pydata.org/en/stable/examples/ROMS_ocean_model.html

Zo_rho = (roms.hc * roms.s_rho + roms.Cs_r * roms.h) / (roms.hc + roms.h)
z_rho = roms.zeta + (roms.zeta + roms.h) * Zo_rho

In [9]:
roms.coords['z_rho'] = z_rho.transpose()   # needing transpose seems to be an xarray bug
roms.temp

<xarray.DataArray 'temp' (ocean_time: 168, s_rho: 40, eta_rho: 63, xi_rho: 76)>
[32175360 values with dtype=float64]
Coordinates:
  * ocean_time  (ocean_time) datetime64[ns] 2007-01-17 2007-02-15 ... 2020-12-16
  * s_rho       (s_rho) float64 -0.9875 -0.9625 -0.9375 ... -0.0375 -0.0125
    lat_rho     (xi_rho, eta_rho) float64 37.18 37.23 37.28 ... 43.11 43.16
    lon_rho     (xi_rho, eta_rho) float64 -73.05 -73.1 -73.15 ... -71.28 -71.33
    z_rho       (s_rho, xi_rho, eta_rho, ocean_time) float64 -2.883e+03 ... -...
Dimensions without coordinates: eta_rho, xi_rho
Attributes:
    long_name:      time-averaged potential temperature
    units:          Celsius
    time:           ocean_time
    field:          temperature, scalar, series
    standard_name:  sea_water_potential_temperature

In [13]:
from datetime import datetime as dt
import time

def toYearFraction(date):
    def sinceEpoch(date): # returns seconds since epoch
        return time.mktime(date.timetuple())
    s = sinceEpoch

    year = date.year
    startOfThisYear = dt(year=year, month=1, day=1)
    startOfNextYear = dt(year=year+1, month=1, day=1)

    yearElapsed = s(date) - s(startOfThisYear)
    yearDuration = s(startOfNextYear) - s(startOfThisYear)
    fraction = yearElapsed/yearDuration

    return date.year + fraction


In [33]:
pandas_dates = pd.to_datetime(roms.ocean_time.values)
new_time = np.empty(len(pandas_dates))
for i in range(len(pandas_dates)):
    new_time[i] = toYearFraction(pandas_dates[i])

In [35]:
roms.coords['new_time'] = new_time

In [36]:
roms

<xarray.Dataset>
Dimensions:     (eta_rho: 63, eta_u: 63, eta_v: 62, new_time: 168, ocean_time: 168, s_rho: 40, xi_rho: 76, xi_u: 76, xi_v: 77)
Coordinates:
  * ocean_time  (ocean_time) datetime64[ns] 2007-01-17 2007-02-15 ... 2020-12-16
  * s_rho       (s_rho) float64 -0.9875 -0.9625 -0.9375 ... -0.0375 -0.0125
    lat_rho     (xi_rho, eta_rho) float64 37.18 37.23 37.28 ... 43.11 43.16
    lon_rho     (xi_rho, eta_rho) float64 -73.05 -73.1 -73.15 ... -71.28 -71.33
    lat_u       (eta_u, xi_u) float64 37.2 37.24 37.28 ... 43.1 43.14 43.18
    lon_u       (eta_u, xi_u) float64 -73.02 -72.95 -72.89 ... -71.36 -71.3
    lat_v       (eta_v, xi_v) float64 37.2 37.24 37.28 ... 43.09 43.13 43.17
    lon_v       (eta_v, xi_v) float64 -73.07 -73.01 -72.95 ... -71.3 -71.24
    z_rho       (s_rho, xi_rho, eta_rho, ocean_time) float64 -2.883e+03 ... -...
  * new_time    (new_time) float64 2.007e+03 2.007e+03 ... 2.021e+03 2.021e+03
Dimensions without coordinates: eta_rho, eta_u, eta_v, xi_rho, xi_u, xi_v
Data variables:
    salt        (ocean_time, s_rho, eta_rho, xi_rho) float64 ...
    Cs_r        (s_rho) float64 -0.9714 -0.9079 ... -0.0001464 -1.619e-05
    zeta        (ocean_time, eta_rho, xi_rho) float64 ...
    h           (eta_rho, xi_rho) float64 2.965e+03 2.983e+03 ... 5.0 5.0
    hc          float64 250.0
    temp        (ocean_time, s_rho, eta_rho, xi_rho) float64 ...
    u           (ocean_time, s_rho, eta_u, xi_u) float64 ...
    v           (ocean_time, s_rho, eta_v, xi_v) float64 ...
Attributes: (12/48)
    file:                       doppio_avg_0366.nc
    format:                     netCDF-4/HDF5 file
    Conventions:                CF-1.4, SGRID-0.3
    type:                       ROMS/TOMS nonlinear model averages file
    title:                      DopAnV2R3-ini2007 - ROMS DOPPIO 7km reanalysi...
    var_info:                   ../Data/varinfo809t_daily.dat
    ...                         ...
    summary:                    doppio
    History:                    Translated to CF-1.0 Conventions by Netcdf-Ja...
    geospatial_lat_min:         37.178910308832386
    geospatial_lat_max:         43.17683923036529
    geospatial_lon_min:         -76.23060799706121
    geospatial_lon_max:         -68.17979698921016

In [38]:
roms.to_netcdf('roms_converted.nc')